In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
import optuna.integration.lightgbm as lgb_o
from itertools import combinations, permutations
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
class Results:
    @staticmethod
    def scrape(race_id_list):
        """
        レース結果データをスクレイピングする関数
        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト
        Returns:
        ----------
        race_results_df : pandas.DataFrame
            全レース結果データをまとめてDataFrame型にしたもの
        """
        #race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
            time.sleep(1)
            try:
                url = "https://db.netkeiba.com/race/" + race_id
                # スクレイピング
                html = requests.get(url)
                html.encoding = "EUC-JP"
                # メインとなるテーブルデータを取得
                df = pd.read_html(html.text)[0]
                # 列名に半角スペースがあれば除去する
                df = df.rename(columns=lambda x: x.replace(' ', ''))
                # 天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                soup = BeautifulSoup(html.text, "html.parser")
                texts = (
                    soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                for text in info:
                    if text in ["芝", "ダート"]:
                        df["race_type"] = [text] * len(df)
                    if "障" in text:
                        df["race_type"] = ["障害"] * len(df)
                    if "m" in text:
                        df["course_len"] = [int(re.findall(r"\d+", text)[-1])] * len(df)
                    if text in ["良", "稍重", "重", "不良"]:
                        df["ground_state"] = [text] * len(df)
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df["weather"] = [text] * len(df)
                    if "年" in text:
                        df["date"] = [text] * len(df)
                #馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])
                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list
                #インデックスをrace_idにする
                df.index = [race_id] * len(df)
                race_results[race_id] = df
            #存在しないrace_idを飛ばす
            except IndexError:
                continue
            except AttributeError: #存在しないrace_idでAttributeErrorになるページもあるので追加
                continue
            #wifiの接続が切れた時などでも途中までのデータを返せるようにする
            except Exception as e:
                print(e)
                break
            #Jupyterで停止ボタンを押した時の対処
            except:
                break
        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])
        return race_results_df

In [27]:
#馬の過去成績データを処理するクラス
class HorseResults:
    @staticmethod
    def scrape(horse_id_list):
        """
        馬の過去成績データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        horse_results_df : pandas.DataFrame
            全馬の過去成績データをまとめてDataFrame型にしたもの
        """

        #horse_idをkeyにしてDataFrame型を格納
        horse_results = {}
        for horse_id in tqdm(horse_id_list):
            time.sleep(1)
            try:
                url = 'https://db.netkeiba.com/horse/' + horse_id
                df = pd.read_html(url)[3]
                #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0]=='受賞歴':
                    df = pd.read_html(url)[4]
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #pd.DataFrame型にして一つのデータにまとめる
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])

        return horse_results_df

In [12]:
race_id_list = []
for place in range(1, 11, 1):
    for kai in range(1, 7, 1):
        for day in range(1, 13, 1):
            for r in range(1, 13, 1):
                race_id = "2019" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)

In [37]:
race_id_list=["202206050911"]
results = Results.scrape(race_id_list)
results.head()

  0%|          | 0/1 [00:00<?, ?it/s]

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,course_len,weather,race_type,ground_state,date,horse_id,jockey_id
202206050911,1,6,11,ドゥラエレーデ,牡2,55,ムルザバ,2:01.5,NaN,90.6,14,508(+4),[西] 池添学,2000,晴,芝,良,2022年12月28日,2020103626,05627
202206050911,2,4,8,トップナイフ,牡2,55,横山典弘,2:01.5,ハナ,18.7,7,494(+4),[西] 昆貢,2000,晴,芝,良,2022年12月28日,2020106402,00660
202206050911,3,7,15,キングズレイン,牡2,55,ルメール,2:01.7,1.1/4,18.2,6,484(+4),[東] 手塚貴久,2000,晴,芝,良,2022年12月28日,2020103466,05339
202206050911,4,1,1,ファントムシーフ,牡2,55,福永祐一,2:01.7,クビ,5.1,2,502(+10),[西] 西村真幸,2000,晴,芝,良,2022年12月28日,2020106582,01014
202206050911,5,8,18,ミッキーカプチーノ,牡2,55,戸崎圭太,2:01.8,1/2,3.0,1,524(+2),[西] 矢作芳人,2000,晴,芝,良,2022年12月28日,2020104484,05386


In [ ]:
horse_id_list = results['horse_id'].unique()
horse_results = HorseResults.scrape(horse_id_list)
horse_results #jupyterで出力

In [43]:
#日付をdatetime型へ変換
horse_results["日付"] = pd.to_datetime(horse_results["日付"], format='%Y/%m/%d')
horse_results = horse_results[horse_results["日付"] < datetime.datetime(2022,12,28)]
horse_results

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
2020103626,2022-11-19,5東京5,晴,11.0,東京スポーツ杯2歳S(G2),NaN,11,6.0,7,20.1,...,0.2,**,2-1-2,35.2-34.9,35.0,504(-4),NaN,NaN,ガストリック,570.0
2020103626,2022-08-20,2札幌3,曇,2.0,2歳未勝利,NaN,12,6.0,7,1.5,...,-0.1,**,2-2-1-1,30.0-37.8,37.8,508(+2),NaN,NaN,(シゲルカミカゼ),520.0
2020103626,2022-08-06,1札幌5,曇,1.0,2歳未勝利,NaN,10,1.0,1,2.5,...,0.3,**,1-1-1-1,36.6-34.7,35.0,506(-4),NaN,NaN,ドゥーラ,210.0
2020103626,2022-06-26,3阪神4,晴,5.0,2歳新馬,NaN,12,5.0,5,2.0,...,0.7,**,3-3,36.2-34.4,35.0,510(0),NaN,NaN,カルロヴェローチェ,70.0
2020106402,2022-11-26,5阪神7,晴,11.0,ラジオN杯京都2歳S(G3),NaN,15,3.0,4,8.6,...,0.0,**,6-6-5-6,35.9-35.7,35.3,490(+6),NaN,NaN,グリューネグリーン,1315.2
2020106402,2022-10-29,4阪神8,晴,9.0,萩S(L),NaN,7,3.0,3,18.0,...,-0.3,**,2-2,35.8-34.3,33.9,484(+2),NaN,NaN,(ナイトキャッスル),1715.4
2020106402,2022-09-24,5中京6,晴,9.0,野路菊S(OP),NaN,7,2.0,2,12.4,...,1.5,**,2-3-3-3,36.3-33.8,34.9,482(-2),NaN,NaN,ファントムシーフ,240.0
2020106402,2022-09-04,2札幌8,晴,1.0,2歳未勝利,NaN,8,1.0,1,7.1,...,-0.7,**,1-1-1-1,36.0-36.7,36.7,484(-6),NaN,NaN,(エルデストサン),520.0
2020106402,2022-08-13,2札幌1,晴,1.0,2歳未勝利,NaN,12,3.0,3,3.4,...,0.4,**,1-1-1-1,36.9-36.3,36.7,490(-10),NaN,NaN,トーセンウォルト,130.0
2020106402,2022-07-24,1札幌2,曇,5.0,2歳新馬,NaN,9,6.0,6,6.8,...,0.6,**,6-4-6-6,38.8-36.0,35.9,500(0),NaN,NaN,ドゥアイズ,NaN
